Next:
t-sne embed and tensorboard visualization
https://github.com/joeddav/blog/blob/master/_notebooks/2020-05-29-ZSL.ipynb
https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b


In [1]:
!pip install transformers

/usr/bin/sh: /lus/scratch/mzaman/anaconda3/lib/libreadline.so.7: no version information available (required by /usr/bin/sh)
/bin/sh: /lus/scratch/mzaman/anaconda3/lib/libreadline.so.7: no version information available (required by /bin/sh)
     |████████████████████████████████| 778kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 8.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/46/19/c5ab91b1b05cfe63cccd5cfc971db9214c6dd6ced54e33c30d5af1d2bc43/packaging-20.4-py2.py3-none-any.whl
     |████████████████████████████████| 3.0MB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 665kB 26.2MB/s eta 0:00:01
     |████████████████████████████████| 890kB 26.6MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d8981a47baddc6fbb3dc16514c5cd6736532a2779af11f08f624a8d16d2affe1
  Stored in directory: /home/users/mzaman/.

In [12]:
[sentence] + labels
inputs
output[:1].shape
sentence_rep.shape

torch.Size([1, 768])

In [2]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

sentence = 'Who are you voting for in 2020?'
labels = ['business', 'art & culture', 'politics']

# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                     return_tensors='pt',
                                     pad_to_max_length=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

# now find the labels with the highest cosine similarities to
# the sentence
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')

/lus/scratch/mzaman/anaconda3/envs/torchclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/lus/scratch/mzaman/anaconda3/envs/torchclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/lus/scratch/mzaman/anaconda3/envs/torchclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/lus/scratch/


label: politics 	 similarity: 0.21561527252197266
label: business 	 similarity: 0.004524129908531904
label: art & culture 	 similarity: -0.02739684097468853


In [4]:
inputs

{'input_ids': tensor([[  101,  2040,  2024,  2017,  6830,  2005,  1999, 12609,  1029,   102],
        [  101,  2449,   102,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2396,  1004,  3226,   102,     0,     0,     0,     0,     0],
        [  101,  4331,   102,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}